# Ensemble models

In [205]:
from utils import data_preparation as prep, data_process as proc
from utils.utils_baselines import *

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score


from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from julia import Julia
Julia(sysimage='/home/gridsan/groups/IAI/images/2.0.0/julia-1.4.2/sys.so', compiled_modules = False)
from interpretableai import iai
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from xgboost import XGBClassifier

In [76]:
def get_baselines_train(x_df, x, test_size = 0.2):
    y_baseline_speed = np.array(x_df['speed_forecast'])
    y_baseline_cos_wind = np.array(x_df['cos_wind_dir_forecast'])
    y_baseline_sin_wind = np.array(x_df['sin_wind_dir_forecast'])
    _, _, y_train_baseline_speed, y_test_baseline_speed = train_test_split(x, y_baseline_speed, test_size = test_size, shuffle = False)
    _, _, y_train_baseline_cos_wind, y_test_baseline_cos_wind = train_test_split(x, y_baseline_cos_wind, test_size = test_size, shuffle = False)
    _, _, y_train_baseline_sin_wind, y_test_baseline_sin_wind = train_test_split(x, y_baseline_sin_wind, test_size = test_size, shuffle = False)
    y_baseline_dangerous_scenarios = get_all_dangerous_scenarios(y_train_baseline_speed, y_train_baseline_cos_wind, y_train_baseline_sin_wind)
    y_baseline_scenarios = get_all_scenarios(y_train_baseline_speed, y_train_baseline_cos_wind, y_train_baseline_sin_wind, b_scenarios=True)
    return y_train_baseline_speed, y_train_baseline_cos_wind, y_train_baseline_sin_wind, y_baseline_dangerous_scenarios, y_baseline_scenarios

In [262]:
# Data

# call data_preparation.py
if True:
    measurement = prep.prepare_measurement()
    forecast = prep.prepare_forecast()

    # keep useful columns
    measurement = measurement[['speed', 'cos_wind_dir', 'sin_wind_dir', 'temp', 'radiation', 'precip', 'season']]

    # call data_process.py

    steps_in = 48
    steps_out = 1

    x_df, y_df, x, y_speed = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'speed')
    _, _, _, y_cos = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'cos_wind_dir')
    _, _, _, y_sin = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'sin_wind_dir')
    y_scenarios = get_all_scenarios(np.array(y_speed), np.array(y_cos), np.array(y_sin), b_scenarios=True)
    y_dangerous = get_all_dangerous_scenarios(np.array(y_speed), np.array(y_cos), np.array(y_sin))
    X_train, X_test, y_train_dangerous, y_test_dangerous = train_test_split(x, y_dangerous, test_size=0.2,
                                                                            shuffle=False)
    _, _, y_train_scenarios, y_test_scenarios = train_test_split(x, y_scenarios, test_size=0.2, shuffle=False)
    #_, _, y_train_speed, y_test_speed = train_test_split(x, y_speed, test_size=0.2, shuffle=False)
    #_, _, y_train_cos, y_test_cos = train_test_split(x, y_cos, test_size=0.2, shuffle=False)
    #_, _, y_train_sin, y_test_sin = train_test_split(x, y_sin, test_size=0.2, shuffle=False)
    #
    #names = list(x_df.columns)
    #names.remove('present_time')
    #names.remove('forecast_time')
    #X_train2 = pd.DataFrame(X_train)
    #X_train2.columns = names
    #(X_train_reg, y_train_speed_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_speed), train_proportion=0.9999)
    #(_, y_train_cos_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_cos), train_proportion=0.9999)
    #(_, y_train_sin_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_sin), train_proportion=0.9999)
    ###BASELINES
    y_test_baseline_speed, y_test_baseline_cos_wind, y_test_baseline_sin_wind, y_test_baseline_dangerous_scenarios, y_test_baseline_scenarios = get_baselines(
        x_df, x)
    y_train_baseline_speed, y_train_baseline_cos_wind, y_train_baseline_sin_wind, y_train_baseline_dangerous_scenarios, y_train_baseline_scenarios = get_baselines_train(
        x_df, x)

read csv semester csv files from 2015s2 to 2020s1
smooth wind direction
generate seasonality categorical feature
reading forecast data
smooth wind direction


In [263]:
path_cos = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_cos_in48_out1_sparsity_0.json'
path_sin = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_sin_in48_out1_sparsity_0.json'
path_speed = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_speed_in48_out1_sparsity_0.json'
path_class = '/home/gridsan/leobix/safi/Trees/vfinal_Classification_tree_dangerous_in48_out1_criterion_gini.json'

lnr_cos = iai.read_json(path_cos)
lnr_sin = iai.read_json(path_sin)
lnr_speed = iai.read_json(path_speed)
lnr_dangerous = iai.read_json(path_class)

y_hat_speed = lnr_speed.predict(X_test)
y_hat_cos = lnr_cos.predict(X_test)
y_hat_sin = lnr_sin.predict(X_test)
y_hat_dangerous_from_regression = get_all_dangerous_scenarios(y_hat_speed, y_hat_cos, y_hat_sin)
y_hat_dangerous_from_classification_proba = np.array(lnr_dangerous.predict_proba(X_test)['1'])
y_hat_dangerous_from_classification = np.array(lnr_dangerous.predict(X_test))

y_hat_speed_train = lnr_speed.predict(X_train)
y_hat_cos_train = lnr_cos.predict(X_train)
y_hat_sin_train = lnr_sin.predict(X_train)
y_hat_dangerous_from_regression_train = get_all_dangerous_scenarios(y_hat_speed_train, y_hat_cos_train, y_hat_sin_train)
y_hat_dangerous_from_classification_proba_train = np.array(lnr_dangerous.predict_proba(X_train)['1'])
y_hat_dangerous_from_classification_train = np.array(lnr_dangerous.predict(X_train))

In [264]:
X_train_ensemble = pd.read_csv('results_ensemble/xgboost_result_train_1.csv?dl=0')
X_test_ensemble = pd.read_csv('results_ensemble/xgboost_result_test_1.csv?dl=0')
X_train_ensemble = X_train_ensemble.drop(['true', 'baseline'], axis = 1)
#X_train_ensemble = X_train_ensemble.drop([0], axis = 0)
X_test_ensemble = X_test_ensemble.drop(['true', 'baseline'], axis = 1)
X_test_ensemble_dt = pd.read_csv('results_ensemble/dt_result_test_1.csv?dl=0') 

In [265]:
X_test_ensemble['OCT_speed'] = y_hat_speed
X_test_ensemble['OCT_cos'] = y_hat_cos
X_test_ensemble['OCT_sin'] = y_hat_sin
X_test_ensemble['OCT_indirect'] = y_hat_dangerous_from_regression
X_test_ensemble['OCT_direct'] = y_hat_dangerous_from_classification
X_test_ensemble['OCT_direct_probas'] = y_hat_dangerous_from_classification_proba
X_test_ensemble['NUMTECH_speed'] = y_test_baseline_speed
X_test_ensemble['NUMTECH_cos'] = y_test_baseline_cos_wind
X_test_ensemble['NUMTECH_sin'] = y_test_baseline_sin_wind
X_test_ensemble['NUMTECH_indirect'] = y_test_baseline_dangerous_scenarios

X_test_ensemble['DT_indirect'] = X_test_ensemble_dt['dangerous_indirect']
X_test_ensemble['DT_direct'] = X_test_ensemble_dt['dangerous']

In [266]:
X_train_ensemble['OCT_speed'] = y_hat_speed_train
X_train_ensemble['OCT_cos'] = y_hat_cos_train
X_train_ensemble['OCT_sin'] = y_hat_sin_train
X_train_ensemble['OCT_indirect'] = y_hat_dangerous_from_regression_train
X_train_ensemble['OCT_direct'] = y_hat_dangerous_from_classification_train
X_train_ensemble['OCT_direct_probas'] = y_hat_dangerous_from_classification_proba_train
X_train_ensemble['NUMTECH_speed'] = y_train_baseline_speed
X_train_ensemble['NUMTECH_cos'] = y_train_baseline_cos_wind
X_train_ensemble['NUMTECH_sin'] = y_train_baseline_sin_wind
X_train_ensemble['NUMTECH_indirect'] = y_train_baseline_dangerous_scenarios

# Majority vote

## Using all models available

In [154]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=2)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[7992  284]
 [  98  120]]
0.6812555085380129
Total cost is  862


In [155]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8093  183]
 [ 133   85]]
0.6653225223230206
Total cost is  765


In [156]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=4)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8213   63]
 [ 157   61]]
0.6717543789386559
Total cost is  597


## Using only best models

In [157]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8209   67]
 [ 148   70]]
0.6907200432196982
Total cost is  578


In [158]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=2)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8024  252]
 [ 105  113]]
0.6829442138661026
Total cost is  819


In [169]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'OCT_indirect','dangerous_indirect']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8213   63]
 [ 152   66]]
0.6837417807862147
Total cost is  582


In [173]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8209   67]
 [ 148   70]]
0.6907200432196982
Total cost is  578


In [198]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8200   76]
 [ 136   82]]
0.7117041769343874
Total cost is  560


In [261]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8200   76]
 [ 138   80]]
0.7074633918047339
Total cost is  566


In [284]:
FP = 2
FN = 3
c = confusion_matrix(y_test_dangerous,X_test_ensemble['NUMTECH_indirect'])
#print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('###### Costs are ######')
print('NUMTECH: ', c[0,1]*FP+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['OCT_indirect'])
print('OCT indirect: ', c[0,1]*FP+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['OCT_direct'])
print('OCT direct: ', c[0,1]*FP+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['dangerous_indirect'])
print('XGB indirect: ', c[0,1]*FP+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['dangerous'])
print('XGB direct: ', c[0,1]*FP+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['DT_indirect'])
print('DT indirect: ', c[0,1]*FP+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['DT_direct'])
print('DT direct: ', c[0,1]*FP+c[1,0]*3)

X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'DT_indirect', 'dangerous_indirect', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print('Majority vote w/o OCT', c[0,1]*FP+c[1,0]*3)

X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'DT_indirect', 'OCT_indirect', 'dangerous_indirect', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=4)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print('Majority vote w/o OCT', c[0,1]*FP+c[1,0]*FN)

###### Costs are ######
NUMTECH:  1057
OCT indirect:  829
OCT direct:  838
XGB indirect:  568
XGB direct:  957
DT indirect:  810
DT direct:  1228
Majority vote w/o OCT 561
Majority vote w/o OCT 557


# T+2

In [210]:
# Data

# call data_preparation.py
if True:
    measurement = prep.prepare_measurement()
    forecast = prep.prepare_forecast()

    # keep useful columns
    measurement = measurement[['speed', 'cos_wind_dir', 'sin_wind_dir', 'temp', 'radiation', 'precip', 'season']]

    # call data_process.py

    steps_in = 48
    steps_out = 2

    x_df, y_df, x, y_speed = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'speed')
    _, _, _, y_cos = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'cos_wind_dir')
    _, _, _, y_sin = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'sin_wind_dir')
    y_scenarios = get_all_scenarios(np.array(y_speed), np.array(y_cos), np.array(y_sin), b_scenarios=True)
    y_dangerous = get_all_dangerous_scenarios(np.array(y_speed), np.array(y_cos), np.array(y_sin))
    X_train, X_test, y_train_dangerous, y_test_dangerous = train_test_split(x, y_dangerous, test_size=0.2,
                                                                            shuffle=False)
    _, _, y_train_scenarios, y_test_scenarios = train_test_split(x, y_scenarios, test_size=0.2, shuffle=False)
    #_, _, y_train_speed, y_test_speed = train_test_split(x, y_speed, test_size=0.2, shuffle=False)
    #_, _, y_train_cos, y_test_cos = train_test_split(x, y_cos, test_size=0.2, shuffle=False)
    #_, _, y_train_sin, y_test_sin = train_test_split(x, y_sin, test_size=0.2, shuffle=False)
    #
    #names = list(x_df.columns)
    #names.remove('present_time')
    #names.remove('forecast_time')
    #X_train2 = pd.DataFrame(X_train)
    #X_train2.columns = names
    #(X_train_reg, y_train_speed_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_speed), train_proportion=0.9999)
    #(_, y_train_cos_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_cos), train_proportion=0.9999)
    #(_, y_train_sin_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_sin), train_proportion=0.9999)
    ###BASELINES
    y_test_baseline_speed, y_test_baseline_cos_wind, y_test_baseline_sin_wind, y_test_baseline_dangerous_scenarios, y_test_baseline_scenarios = get_baselines(
        x_df, x)
    y_train_baseline_speed, y_train_baseline_cos_wind, y_train_baseline_sin_wind, y_train_baseline_dangerous_scenarios, y_train_baseline_scenarios = get_baselines_train(
        x_df, x)
    


read csv semester csv files from 2015s2 to 2020s1
smooth wind direction
generate seasonality categorical feature
reading forecast data
smooth wind direction


In [211]:
path_cos = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_cos_in48_out2_sparsity_0.json'
path_sin = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_sin_in48_out2_sparsity_0.json'
path_speed = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_speed_in48_out2_sparsity_0.json'
path_class = '/home/gridsan/leobix/safi/Trees/vfinal_Classification_tree_dangerous_in48_out2_criterion_gini.json'

lnr_cos = iai.read_json(path_cos)
lnr_sin = iai.read_json(path_sin)
lnr_speed = iai.read_json(path_speed)
lnr_dangerous = iai.read_json(path_class)

y_hat_speed = lnr_speed.predict(X_test)
y_hat_cos = lnr_cos.predict(X_test)
y_hat_sin = lnr_sin.predict(X_test)
y_hat_dangerous_from_regression = get_all_dangerous_scenarios(y_hat_speed, y_hat_cos, y_hat_sin)
y_hat_dangerous_from_classification_proba = np.array(lnr_dangerous.predict_proba(X_test)['1'])
y_hat_dangerous_from_classification = np.array(lnr_dangerous.predict(X_test))

y_hat_speed_train = lnr_speed.predict(X_train)
y_hat_cos_train = lnr_cos.predict(X_train)
y_hat_sin_train = lnr_sin.predict(X_train)
y_hat_dangerous_from_regression_train = get_all_dangerous_scenarios(y_hat_speed_train, y_hat_cos_train, y_hat_sin_train)
y_hat_dangerous_from_classification_proba_train = np.array(lnr_dangerous.predict_proba(X_train)['1'])
y_hat_dangerous_from_classification_train = np.array(lnr_dangerous.predict(X_train))

In [212]:
X_train_ensemble = pd.read_csv('results_ensemble/xgboost_result_train_2.csv?dl=0')
X_test_ensemble = pd.read_csv('results_ensemble/xgboost_result_test_2.csv?dl=0')
X_train_ensemble = X_train_ensemble.drop(['true', 'baseline'], axis = 1)
#X_train_ensemble = X_train_ensemble.drop([0], axis = 0)
X_test_ensemble = X_test_ensemble.drop(['true', 'baseline'], axis = 1)

In [213]:
X_test_ensemble['OCT_speed'] = y_hat_speed
X_test_ensemble['OCT_cos'] = y_hat_cos
X_test_ensemble['OCT_sin'] = y_hat_sin
X_test_ensemble['OCT_indirect'] = y_hat_dangerous_from_regression
X_test_ensemble['OCT_direct'] = y_hat_dangerous_from_classification
X_test_ensemble['OCT_direct_probas'] = y_hat_dangerous_from_classification_proba
X_test_ensemble['NUMTECH_speed'] = y_test_baseline_speed
X_test_ensemble['NUMTECH_cos'] = y_test_baseline_cos_wind
X_test_ensemble['NUMTECH_sin'] = y_test_baseline_sin_wind
X_test_ensemble['NUMTECH_indirect'] = y_test_baseline_dangerous_scenarios

In [214]:
X_train_ensemble['OCT_speed'] = y_hat_speed_train
X_train_ensemble['OCT_cos'] = y_hat_cos_train
X_train_ensemble['OCT_sin'] = y_hat_sin_train
X_train_ensemble['OCT_indirect'] = y_hat_dangerous_from_regression_train
X_train_ensemble['OCT_direct'] = y_hat_dangerous_from_classification_train
X_train_ensemble['OCT_direct_probas'] = y_hat_dangerous_from_classification_proba_train
X_train_ensemble['NUMTECH_speed'] = y_train_baseline_speed
X_train_ensemble['NUMTECH_cos'] = y_train_baseline_cos_wind
X_train_ensemble['NUMTECH_sin'] = y_train_baseline_sin_wind
X_train_ensemble['NUMTECH_indirect'] = y_train_baseline_dangerous_scenarios

In [216]:
c = confusion_matrix(y_test_dangerous,X_test_ensemble['NUMTECH_indirect'])
#print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('###### Costs are ######')
print('NUMTECH: ', c[0,1]*2+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['OCT_indirect'])
print('OCT indirect: ', c[0,1]*2+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['OCT_direct'])
print('OCT direct: ', c[0,1]*2+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['dangerous_indirect'])
print('XGB indirect: ', c[0,1]*2+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['dangerous'])
print('XGB direct: ', c[0,1]*2+c[1,0]*3)

X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print('Majority vote ', c[0,1]*2+c[1,0]*3)

###### Costs are ######
NUMTECH:  1059
OCT indirect:  728
OCT direct:  914
XGB indirect:  614
XGB direct:  908
Majority vote  601


In [246]:
X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print(c)
print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('Total cost is ', c[0,1]*2+c[1,0]*3)

[[8217   59]
 [ 161   57]]
0.664053662819671
Total cost is  601


# T+3

In [247]:
# Data

# call data_preparation.py
if True:
    measurement = prep.prepare_measurement()
    forecast = prep.prepare_forecast()

    # keep useful columns
    measurement = measurement[['speed', 'cos_wind_dir', 'sin_wind_dir', 'temp', 'radiation', 'precip', 'season']]

    # call data_process.py

    steps_in = 48
    steps_out = 3

    x_df, y_df, x, y_speed = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'speed')
    _, _, _, y_cos = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'cos_wind_dir')
    _, _, _, y_sin = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'sin_wind_dir')
    y_scenarios = get_all_scenarios(np.array(y_speed), np.array(y_cos), np.array(y_sin), b_scenarios=True)
    y_dangerous = get_all_dangerous_scenarios(np.array(y_speed), np.array(y_cos), np.array(y_sin))
    X_train, X_test, y_train_dangerous, y_test_dangerous = train_test_split(x, y_dangerous, test_size=0.2,
                                                                            shuffle=False)
    _, _, y_train_scenarios, y_test_scenarios = train_test_split(x, y_scenarios, test_size=0.2, shuffle=False)
    #_, _, y_train_speed, y_test_speed = train_test_split(x, y_speed, test_size=0.2, shuffle=False)
    #_, _, y_train_cos, y_test_cos = train_test_split(x, y_cos, test_size=0.2, shuffle=False)
    #_, _, y_train_sin, y_test_sin = train_test_split(x, y_sin, test_size=0.2, shuffle=False)
    #
    #names = list(x_df.columns)
    #names.remove('present_time')
    #names.remove('forecast_time')
    #X_train2 = pd.DataFrame(X_train)
    #X_train2.columns = names
    #(X_train_reg, y_train_speed_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_speed), train_proportion=0.9999)
    #(_, y_train_cos_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_cos), train_proportion=0.9999)
    #(_, y_train_sin_reg), _ = iai.split_data('regression', X_train2, np.array(y_train_sin), train_proportion=0.9999)
    ###BASELINES
    y_test_baseline_speed, y_test_baseline_cos_wind, y_test_baseline_sin_wind, y_test_baseline_dangerous_scenarios, y_test_baseline_scenarios = get_baselines(
        x_df, x)
    y_train_baseline_speed, y_train_baseline_cos_wind, y_train_baseline_sin_wind, y_train_baseline_dangerous_scenarios, y_train_baseline_scenarios = get_baselines_train(
        x_df, x)
    



read csv semester csv files from 2015s2 to 2020s1
smooth wind direction
generate seasonality categorical feature
reading forecast data
smooth wind direction


In [248]:
path_cos = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_cos_in48_out3_sparsity_0.json'
path_sin = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_sin_in48_out3_sparsity_0.json'
path_speed = '/home/gridsan/leobix/safi/Trees3/vfinal_Regression_tree_speed_in48_out3_sparsity_0.json'
path_class = '/home/gridsan/leobix/safi/Trees/vfinal_Classification_tree_dangerous_in48_out3_criterion_gini.json'

lnr_cos = iai.read_json(path_cos)
lnr_sin = iai.read_json(path_sin)
lnr_speed = iai.read_json(path_speed)
lnr_dangerous = iai.read_json(path_class)

y_hat_speed = lnr_speed.predict(X_test)
y_hat_cos = lnr_cos.predict(X_test)
y_hat_sin = lnr_sin.predict(X_test)
y_hat_dangerous_from_regression = get_all_dangerous_scenarios(y_hat_speed, y_hat_cos, y_hat_sin)
y_hat_dangerous_from_classification_proba = np.array(lnr_dangerous.predict_proba(X_test)['1'])
y_hat_dangerous_from_classification = np.array(lnr_dangerous.predict(X_test))

y_hat_speed_train = lnr_speed.predict(X_train)
y_hat_cos_train = lnr_cos.predict(X_train)
y_hat_sin_train = lnr_sin.predict(X_train)
y_hat_dangerous_from_regression_train = get_all_dangerous_scenarios(y_hat_speed_train, y_hat_cos_train, y_hat_sin_train)
y_hat_dangerous_from_classification_proba_train = np.array(lnr_dangerous.predict_proba(X_train)['1'])
y_hat_dangerous_from_classification_train = np.array(lnr_dangerous.predict(X_train))

In [250]:
X_train_ensemble = pd.read_csv('results_ensemble/xgboost_result_train_3.csv?dl=0')
X_test_ensemble = pd.read_csv('results_ensemble/xgboost_result_test_3.csv?dl=0')
X_train_ensemble = X_train_ensemble.drop(['true', 'baseline'], axis = 1)
#X_train_ensemble = X_train_ensemble.drop([0], axis = 0)
X_test_ensemble = X_test_ensemble.drop(['true', 'baseline'], axis = 1)

In [251]:
X_test_ensemble['OCT_speed'] = y_hat_speed
X_test_ensemble['OCT_cos'] = y_hat_cos
X_test_ensemble['OCT_sin'] = y_hat_sin
X_test_ensemble['OCT_indirect'] = y_hat_dangerous_from_regression
X_test_ensemble['OCT_direct'] = y_hat_dangerous_from_classification
X_test_ensemble['OCT_direct_probas'] = y_hat_dangerous_from_classification_proba
X_test_ensemble['NUMTECH_speed'] = y_test_baseline_speed
X_test_ensemble['NUMTECH_cos'] = y_test_baseline_cos_wind
X_test_ensemble['NUMTECH_sin'] = y_test_baseline_sin_wind
X_test_ensemble['NUMTECH_indirect'] = y_test_baseline_dangerous_scenarios

In [252]:
X_train_ensemble['OCT_speed'] = y_hat_speed_train
X_train_ensemble['OCT_cos'] = y_hat_cos_train
X_train_ensemble['OCT_sin'] = y_hat_sin_train
X_train_ensemble['OCT_indirect'] = y_hat_dangerous_from_regression_train
X_train_ensemble['OCT_direct'] = y_hat_dangerous_from_classification_train
X_train_ensemble['OCT_direct_probas'] = y_hat_dangerous_from_classification_proba_train
X_train_ensemble['NUMTECH_speed'] = y_train_baseline_speed
X_train_ensemble['NUMTECH_cos'] = y_train_baseline_cos_wind
X_train_ensemble['NUMTECH_sin'] = y_train_baseline_sin_wind
X_train_ensemble['NUMTECH_indirect'] = y_train_baseline_dangerous_scenarios

In [253]:
c = confusion_matrix(y_test_dangerous,X_test_ensemble['NUMTECH_indirect'])
#print(f1_score(y_test_dangerous, consensus_majority, average = 'macro'))
print('###### Costs are ######')
print('NUMTECH: ', c[0,1]*2+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['OCT_indirect'])
print('OCT indirect: ', c[0,1]*2+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['OCT_direct'])
print('OCT direct: ', c[0,1]*2+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['dangerous_indirect'])
print('XGB indirect: ', c[0,1]*2+c[1,0]*3)

c = confusion_matrix(y_test_dangerous,X_test_ensemble['dangerous'])
print('XGB direct: ', c[0,1]*2+c[1,0]*3)

X_test_majority_vote = X_test_ensemble[['NUMTECH_indirect', 'OCT_direct', 'dangerous_indirect', 'dangerous_indirect', 'dangerous']]
consensus_majority = np.array(X_test_majority_vote.sum(axis = 1) >=3)*1
c = confusion_matrix(y_test_dangerous,consensus_majority)
print('Majority vote ', c[0,1]*2+c[1,0]*3)

###### Costs are ######
NUMTECH:  1067
OCT indirect:  691
OCT direct:  848
XGB indirect:  591
XGB direct:  847
Majority vote  582


# XGB consensus

In [203]:
# Set the parameters by cross-validation
tuned_parameters = [{'penalty': ['l2', 'l1'], 'C': [1, 2, 0.1, 0.05, 0.2, 0.3, 10]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        LogisticRegression(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train_ensemble, y_train_dangerous)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test_dangerous, clf.predict(X_test_ensemble)
    print(classification_report(y_true, y_pred))
    c = confusion_matrix(y_test_dangerous,y_pred)
    print('Ensemble ', c[0,1]*2+c[1,0]*3)

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 0.3, 'penalty': 'l1'}

Grid scores on development set:

0.951 (+/-0.062) for {'C': 1, 'penalty': 'l2'}
0.950 (+/-0.055) for {'C': 1, 'penalty': 'l1'}
0.952 (+/-0.059) for {'C': 2, 'penalty': 'l2'}
0.949 (+/-0.052) for {'C': 2, 'penalty': 'l1'}
0.940 (+/-0.074) for {'C': 0.1, 'penalty': 'l2'}
0.951 (+/-0.055) for {'C': 0.1, 'penalty': 'l1'}
0.938 (+/-0.075) for {'C': 0.05, 'penalty': 'l2'}
0.948 (+/-0.071) for {'C': 0.05, 'penalty': 'l1'}
0.943 (+/-0.070) for {'C': 0.2, 'penalty': 'l2'}
0.951 (+/-0.056) for {'C': 0.2, 'penalty': 'l1'}
0.946 (+/-0.068) for {'C': 0.3, 'penalty': 'l2'}
0.953 (+/-0.056) for {'C': 0.3, 'penalty': 'l1'}
0.949 (+/-0.055) for {'C': 10, 'penalty': 'l2'}
0.948 (+/-0.052) for {'C': 10, 'penalty': 'l1'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  

In [206]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'max_depth': [2,3,4,5], 'learning_rate': [0.3, 0.2, 0.15]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        XGBClassifier(tree_method='gpu_hist', gpu_id=0), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train_ensemble, y_train_dangerous)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    y_true, y_pred = y_test_dangerous, clf.predict(X_test_ensemble)
    print(classification_report(y_true, y_pred))
    c = confusion_matrix(y_test_dangerous,y_pred)
    print('Ensemble ', c[0,1]*2+c[1,0]*3)

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'learning_rate': 0.3, 'max_depth': 2}

Grid scores on development set:

0.965 (+/-0.044) for {'learning_rate': 0.3, 'max_depth': 2}
0.961 (+/-0.041) for {'learning_rate': 0.3, 'max_depth': 3}
0.959 (+/-0.047) for {'learning_rate': 0.3, 'max_depth': 4}
0.958 (+/-0.050) for {'learning_rate': 0.3, 'max_depth': 5}
0.964 (+/-0.052) for {'learning_rate': 0.2, 'max_depth': 2}
0.963 (+/-0.045) for {'learning_rate': 0.2, 'max_depth': 3}
0.963 (+/-0.046) for {'learning_rate': 0.2, 'max_depth': 4}
0.960 (+/-0.047) for {'learning_rate': 0.2, 'max_depth': 5}
0.964 (+/-0.054) for {'learning_rate': 0.15, 'max_depth': 2}
0.964 (+/-0.047) for {'learning_rate': 0.15, 'max_depth': 3}
0.964 (+/-0.046) for {'learning_rate': 0.15, 'max_depth': 4}
0.962 (+/-0.051) for {'learning_rate': 0.15, 'max_depth': 5}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the 

In [208]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'max_depth': [2,3,4,5], 'learning_rate': [0.3, 0.2, 0.15]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        XGBClassifier(tree_method='gpu_hist', gpu_id=0), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_test_ensemble, y_test_dangerous)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    y_true, y_pred = y_test_dangerous, clf.predict(X_test_ensemble)
    print(classification_report(y_true, y_pred))
    c = confusion_matrix(y_test_dangerous,y_pred)
    print('Ensemble ', c[0,1]*2+c[1,0]*3)

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'learning_rate': 0.15, 'max_depth': 2}

Grid scores on development set:

0.804 (+/-0.116) for {'learning_rate': 0.3, 'max_depth': 2}
0.785 (+/-0.139) for {'learning_rate': 0.3, 'max_depth': 3}
0.790 (+/-0.164) for {'learning_rate': 0.3, 'max_depth': 4}
0.771 (+/-0.150) for {'learning_rate': 0.3, 'max_depth': 5}
0.827 (+/-0.119) for {'learning_rate': 0.2, 'max_depth': 2}
0.783 (+/-0.111) for {'learning_rate': 0.2, 'max_depth': 3}
0.795 (+/-0.182) for {'learning_rate': 0.2, 'max_depth': 4}
0.782 (+/-0.186) for {'learning_rate': 0.2, 'max_depth': 5}
0.844 (+/-0.159) for {'learning_rate': 0.15, 'max_depth': 2}
0.797 (+/-0.125) for {'learning_rate': 0.15, 'max_depth': 3}
0.795 (+/-0.131) for {'learning_rate': 0.15, 'max_depth': 4}
0.789 (+/-0.171) for {'learning_rate': 0.15, 'max_depth': 5}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the

In [207]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'hidden_layer_sizes': [(16, 8),(16),(32),(8),(32,8)], 'alpha': [0.001]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        MLPClassifier(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train_ensemble, y_train_dangerous)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    y_true, y_pred = y_test_dangerous, clf.predict(X_test_ensemble)
    print(classification_report(y_true, y_pred))
    c = confusion_matrix(y_test_dangerous,y_pred)
    print('Ensemble ', c[0,1]*2+c[1,0]*3)

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'alpha': 0.001, 'hidden_layer_sizes': (16, 8)}

Grid scores on development set:

0.955 (+/-0.034) for {'alpha': 0.001, 'hidden_layer_sizes': (16, 8)}
0.947 (+/-0.075) for {'alpha': 0.001, 'hidden_layer_sizes': 16}
0.949 (+/-0.049) for {'alpha': 0.001, 'hidden_layer_sizes': 32}
0.945 (+/-0.056) for {'alpha': 0.001, 'hidden_layer_sizes': 8}
0.949 (+/-0.047) for {'alpha': 0.001, 'hidden_layer_sizes': (32, 8)}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      8276
           1       0.30      0.50      0.38       218

   micro avg       0.96      0.96      0.96      8494
   macro avg       0.64      0.74      0.68      8494
weighted avg       0.97      0.96      0.96      8494

Ensemble  830
# Tuning hyper-parameter

In [ ]:
print(lnr_dangerous.score(X_test, y_test_dangerous, criterion='misclassification'))
preds = lnr_dangerous.predict(X_test)
print(confusion_matrix(y_test_dangerous,preds))
print(f1_score(y_test_dangerous, preds, average = 'macro'))